<a href="https://colab.research.google.com/github/A-braxas/Dive-into-deep-learning/blob/main/%E5%8A%A8%E6%89%8B%E5%AD%A6%E6%B7%B1%E5%BA%A6%E5%AD%A6%E4%B9%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 配置环境

In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Dive into deep learning")

In [ ]:
!pwd

/content/gdrive/My Drive/Colab Notebooks/Dive into deep learning


In [ ]:
!pip install -q mxnet-cu101

     |███████████████████████████████▌| 834.1MB 1.9MB/s eta 0:00:07tcmalloc: large alloc 1147494400 bytes == 0x55be1d23e000 @  0x7f256a2dc615 0x55bde374506c 0x55bde3824eba 0x55bde3747e8d 0x55bde383999d 0x55bde37bbfe9 0x55bde37b6b0e 0x55bde374977a 0x55bde37bbe50 0x55bde37b6b0e 0x55bde374977a 0x55bde37b886a 0x55bde383a7c6 0x55bde37b7ee2 0x55bde383a7c6 0x55bde37b7ee2 0x55bde383a7c6 0x55bde37b7ee2 0x55bde383a7c6 0x55bde37b7ee2 0x55bde374969a 0x55bde37b7c9e 0x55bde37b6e0d 0x55bde374977a 0x55bde37b7a45 0x55bde374969a 0x55bde37b7a45 0x55bde37b6b0e 0x55bde374977a 0x55bde37b886a 0x55bde37b6b0e
     |████████████████████████████████| 846.4MB 19kB/s 


In [ ]:
import mxnet as mx
from mxnet import nd, autograd, gluon

# 程序

In [ ]:
from IPython import display
from matplotlib import pyplot as plt
from mxnet import autograd, nd
import random

In [ ]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = nd.random.normal(scale=1, shape=(num_examples, num_inputs))
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += nd.random.normal(scale=0.01, shape=labels.shape)

In [27]:
features[0], labels[0]

(
 [2.2122064 0.7740038]
 <NDArray 2 @cpu(0)>, 
 [6.000587]
 <NDArray 1 @cpu(0)>)

In [30]:
def use_svg_display():
    # 用矢量图显示
    display.set_matplotlib_formats('svg')

def set_figsize(figsize=(3.5, 2.5)):
    use_svg_display()
    # 设置图的尺寸
    plt.rcParams['figure.figsize'] = figsize

def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)  # 样本的读取顺序是随机的
    for i in range(0, num_examples, batch_size):
        j = nd.array(indices[i: min(i + batch_size, num_examples)])
        yield features.take(j), labels.take(j)  # take函数根据索引返回对应元素
#set_figsize()
#plt.scatter(features[:, 1].asnumpy(), labels.asnumpy(), 1);  # 加分号只显示图

In [36]:
batch_size = 10
w = nd.random.normal(scale=0.01, shape=(num_inputs, 1))
b = nd.zeros(shape=(1,))
w.attach_grad()
b.attach_grad()

In [32]:
def linreg(X, w, b):  # 本函数已保存在d2lzh包中方便以后使用
    return nd.dot(X, w) + b

In [33]:
def squared_loss(y_hat, y):  # 本函数已保存在d2lzh包中方便以后使用
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

In [34]:
def sgd(params, lr, batch_size):  # 本函数已保存在d2lzh包中方便以后使用
    for param in params:
        param[:] = param - lr * param.grad / batch_size

In [37]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

for epoch in range(num_epochs):  # 训练模型一共需要num_epochs个迭代周期
    # 在每一个迭代周期中，会使用训练数据集中所有样本一次（假设样本数能够被批量大小整除）。X
    # 和y分别是小批量样本的特征和标签
    for X, y in data_iter(batch_size, features, labels):
        with autograd.record():
            l = loss(net(X, w, b), y)  # l是有关小批量X和y的损失
        l.backward()  # 小批量的损失对模型参数求梯度
        sgd([w, b], lr, batch_size)  # 使用小批量随机梯度下降迭代模型参数
    train_l = loss(net(features, w, b), labels)
    print('epoch %d, loss %f' % (epoch + 1, train_l.mean().asnumpy()))

epoch 1, loss 0.040706
epoch 2, loss 0.000158
epoch 3, loss 0.000050


In [42]:
true_w, w

([2, -3.4], 
 [[ 1.9998305]
  [-3.3996658]]
 <NDArray 2x1 @cpu(0)>)

In [43]:
true_b, b

(4.2, 
 [4.199946]
 <NDArray 1 @cpu(0)>)